<a href="https://colab.research.google.com/github/vinismachadoo/web_scrapping_farm/blob/main/catalogo_farm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('Hello World')

Hello World
